In [1]:
import numpy as np
import torch as th
import torch.nn as nn
import os
import json
import pandas as pd
import torchvision.transforms as T

from train.multi_hyperparameter import MultiHyperparameter
from custom_models.unet_original import UNet, UNetSmall, UNetLarge
from evaluate.cross_evaluator import SemanticCrossEvaluator
from train.unet_trainer import UnetTrainer
from datasets.semantic_dataset import SemanticDataset
from preprocessing.data_augment import DataAugmenter

C:\Users\silva\anaconda3\envs\TorchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cwd = os.getcwd()
ds1_path = os.path.join(cwd, '../data/cell_type_1')
ds2_path = os.path.join(cwd, '../data/cell_type_2')

cv_param = {'interval_img_out': 1,
            'num_images': 3,
            'device': th.device("cuda" if th.cuda.is_available() else "cpu"),
            'datasets_path': [ds1_path, ds2_path],
            'results_path': os.path.join(cwd, '../results'),
            'folds': [0, 1, 2, 3],
            'epochs_cv': 0,
            'epochs_ct': 100,
            'num_random_params': 3}

param = {'id': 0,
         'padding_mode': 'reflect',
         'out_classes': 2,
         'criterion': nn.CrossEntropyLoss(),
         'optimizer': MultiHyperparameter({
             'type': [
                 # 'sgd',
                 # 'adam',
                 'rmsprop',
                 # 'asgd'
             ],
             'lr_factor': [
                 #1.25,
                 1,
                 #0.75
             ],
             'weight_decay': [
                 # 0,
                 1e-5
             ]
         }).get_full_grid_params(),
         'augment_transform': [{'rotate': False, 'mirror': False, 'translate': False, 'pad': 0},
                               #{'rotate': True, 'mirror': True, 'translate': False, 'pad': 0},
                               #{'rotate': True, 'mirror': True, 'translate': True, 'pad': 16},
                               #{'rotate': True, 'mirror': True, 'translate': True, 'pad': 8},
                               #{'rotate': True, 'mirror': True, 'translate': True, 'pad': 4},
                               #{'rotate': True, 'mirror': True, 'translate': True, 'pad': 2},
                               ],
         'num_augments': 100,
         'binarizer_lr': 0.1,
         'batch_size': 2}

unet_hyps = MultiHyperparameter(param)
params = unet_hyps.get_full_grid_params(indexed=True)
print(len(params))
unet = UNet.__new__(UNet)

1


In [3]:
cte = SemanticCrossEvaluator(unet, cv_param, 3)
print(params)
#report = cte.cross_test_model(params, cv_param['epochs_ct'], cv_param['epochs_cv'])
ds1_losses, ds1_accs, _ = cte.train_validate(params[0], ds1_path, [0, 1, 2], [3], 100)
ds2_losses, ds2_accs, _ = cte.train_validate(params[0], ds2_path, [0, 1, 2], [3], 100)
dsc_losses =  (np.array(ds2_losses) +  np.array(ds1_losses)) / 2
dsc_accs = (np.array(ds2_accs) +  np.array(ds1_accs)) / 2
print(ds1_losses, ds1_accs)
report = pd.DataFrame(data={'ds_0_train': ds1_losses,
                            'ds_0_validate': ds1_accs,
                            'ds_1_train': ds2_accs,
                            'ds_1_validate': ds2_accs,
                            'combined_train': dsc_losses,
                            'combined_validate': dsc_accs})
print(report)
report.to_csv('../results/rms_results_lr2/2_test.csv')



[{'id': 0, 'padding_mode': 'reflect', 'out_classes': 2, 'criterion': CrossEntropyLoss(), 'optimizer': {'type': 'rmsprop', 'lr_factor': 1, 'weight_decay': 1e-05}, 'augment_transform': {'rotate': False, 'mirror': False, 'translate': False, 'pad': 0}, 'num_augments': 100, 'binarizer_lr': 0.1, 'batch_size': 2}]


Training:   1%|▎                           | 1/100 [00:24<39:44, 24.09s/it]

Loss: 0.5448620611429215 Acc: 0.021788280457258224


Training:   2%|▌                           | 2/100 [00:45<37:02, 22.68s/it]

Loss: 0.4026899492740631 Acc: 0.022509649395942688


...

Training:  99%|██████████████████████████▋| 99/100 [37:16<00:22, 22.30s/it]

Loss: 0.3153378641605377 Acc: 0.49680981040000916


Training: 100%|██████████████████████████| 100/100 [37:38<00:00, 22.59s/it]

Loss: 0.3153590416908264 Acc: 0.5105055570602417
[0.5448620611429215, 0.4026899492740631, 0.35280403554439543, 0.33657577753067014, 0.3286727011203766, 0.32749628722667695, 0.32514034271240233, 0.32359801888465883, 0.3212926608324051, 0.3212985122203827, 0.3205341249704361, 0.3198747682571411, 0.32008347630500794, 0.31942833483219146, 0.3201061517000198, 0.31948485016822814, 0.3196733641624451, 0.3181417393684387, 0.31867025077342986, 0.31771185219287873, 0.31810045421123506, 0.31772865176200865, 0.3181619846820831, 0.3170831769704819, 0.31761459887027743, 0.3180161076784134, 0.317661857008934, 0.31769380629062655, 0.3164456671476364, 0.3169369196891785, 0.3177588838338852, 0.31676641285419466, 0.31647180557250976, 0.31628995776176455, 0.3170669096708298, 0.3174820637702942, 0.31591677784919736, 0.3163689148426056, 0.3159127014875412, 0.31601212441921234, 0.3161344265937805, 0.31602152466773986, 0.31681916534900667, 0.316086106300354, 0.315734703540802, 0.3154342019557953, 0.3159074050

OSError: Cannot save file into a non-existent directory: '..\results\rms_results_lr2'

In [4]:
print(report)
report.to_csv('../results/rms_2/4_test.csv')

    ds_0_train  ds_0_validate  ds_1_train  ds_1_validate  combined_train  \
0     0.544862       0.021788    0.508865       0.508865        0.567036   
1     0.402690       0.022510    0.545835       0.545835        0.417714   
2     0.352804       0.574789    0.567389       0.567389        0.362202   
3     0.336576       0.594476    0.584114       0.584114        0.343489   
4     0.328673       0.598828    0.570514       0.570514        0.334447   
..         ...            ...         ...            ...             ...   
95    0.314207       0.576764    0.513990       0.513990        0.314837   
96    0.314267       0.593063    0.533868       0.533868        0.314829   
97    0.314128       0.533076    0.552080       0.552080        0.314702   
98    0.314030       0.575655    0.496810       0.496810        0.314684   
99    0.314233       0.584942    0.510506       0.510506        0.314796   

    combined_validate  
0            0.265327  
1            0.284172  
2            0.

In [ ]:
# dsc_losses =  (np.array(ds2_losses) +  np.array(ds1_losses)) / 2
# dsc_accs = (np.array(ds2_accs) +  np.array(ds1_accs)) / 2
# print(ds1_losses, ds1_accs)
# report = pd.DataFrame(data={'ds_0_train': ds1_losses,
#                             'ds_0_validate': ds1_accs,
#                             'ds_1_train': ds2_accs,
#                             'ds_1_validate': ds2_accs,
#                             'combined_train': dsc_losses,
#                             'combined_validate': dsc_accs})
# print(report)
# report.to_csv('../results/adam_results2/test.csv')